# Day 14: Loyalty Program's Impact on Transaction Patterns

You are a Business Analyst on the Starbucks Rewards team investigating customer transaction behavior. Your team wants to understand how loyalty program membership influences purchasing patterns. The goal is to compare transaction metrics between loyalty members and non-members.

In [ ]:
import pandas as pd
import numpy as np

dim_customers_data = [
  {
    "customer_id": 1,
    "is_loyalty_member": 1
  },
  {
    "customer_id": 2,
    "is_loyalty_member": 0
  },
  {
    "customer_id": 3,
    "is_loyalty_member": 1
  },
  {
    "customer_id": 4,
    "is_loyalty_member": 0
  },
  {
    "customer_id": 5,
    "is_loyalty_member": 1
  },
  {
    "customer_id": 6,
    "is_loyalty_member": 1
  },
  {
    "customer_id": 7,
    "is_loyalty_member": 0
  },
  {
    "customer_id": 8,
    "is_loyalty_member": 1
  },
  {
    "customer_id": 9,
    "is_loyalty_member": 0
  },
  {
    "customer_id": 10,
    "is_loyalty_member": 0
  },
  {
    "customer_id": 11,
    "is_loyalty_member": 1
  },
  {
    "customer_id": 12,
    "is_loyalty_member": 0
  }
]
dim_customers = pd.DataFrame(dim_customers_data)

fct_transactions_data = [
  {
    "customer_id": 1,
    "transaction_id": 101,
    "transaction_date": "2024-07-05",
    "transaction_value": 5.5
  },
  {
    "customer_id": 1,
    "transaction_id": 102,
    "transaction_date": "2024-07-15",
    "transaction_value": 7.25
  },
  {
    "customer_id": 2,
    "transaction_id": 103,
    "transaction_date": "2024-07-10",
    "transaction_value": 4
  },
  {
    "customer_id": 3,
    "transaction_id": 104,
    "transaction_date": "2024-07-20",
    "transaction_value": 8.75
  },
  {
    "customer_id": 4,
    "transaction_id": 105,
    "transaction_date": "2024-07-03",
    "transaction_value": 6.5
  },
  {
    "customer_id": 5,
    "transaction_id": 106,
    "transaction_date": "2024-07-22",
    "transaction_value": 9
  },
  {
    "customer_id": 6,
    "transaction_id": 107,
    "transaction_date": "2024-07-11",
    "transaction_value": 10.5
  },
  {
    "customer_id": 7,
    "transaction_id": 108,
    "transaction_date": "2024-07-18",
    "transaction_value": 4.25
  },
  {
    "customer_id": 8,
    "transaction_id": 109,
    "transaction_date": "2024-07-25",
    "transaction_value": 12
  },
  {
    "customer_id": 9,
    "transaction_id": 110,
    "transaction_date": "2024-07-07",
    "transaction_value": 3.75
  },
  {
    "customer_id": 10,
    "transaction_id": 111,
    "transaction_date": "2024-07-12",
    "transaction_value": 5
  },
  {
    "customer_id": 11,
    "transaction_id": 112,
    "transaction_date": "2024-07-27",
    "transaction_value": 11.25
  },
  {
    "customer_id": 12,
    "transaction_id": 113,
    "transaction_date": "2024-07-08",
    "transaction_value": 6
  },
  {
    "customer_id": 3,
    "transaction_id": 114,
    "transaction_date": "2024-07-30",
    "transaction_value": 7.5
  },
  {
    "customer_id": 5,
    "transaction_id": 115,
    "transaction_date": "2024-07-29",
    "transaction_value": 10
  },
  {
    "customer_id": 1,
    "transaction_id": 116,
    "transaction_date": "2024-07-31",
    "transaction_value": 6.25
  }
]
fct_transactions = pd.DataFrame(fct_transactions_data)


## Question 1

For the month of July 2024, how many transactions did loyalty program members and non-members make? Compare the transaction counts between these two groups.

In [ ]:
# Ensure transaction_date is a datetime type
fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'])

# Filter transactions for July 2024
july_2024_txn = fct_transactions[
    (fct_transactions['transaction_date'].dt.year == 2024) &
    (fct_transactions['transaction_date'].dt.month == 7)
]

# Merge with customer data to get loyalty member info
merged_df = july_2024_txn.merge(
    dim_customers,
    on='customer_id',
    how='left'
)

# Group by loyalty membership and count transactions
txn_counts = merged_df.groupby('is_loyalty_member')['transaction_id'].count().reset_index()

# Rename columns for clarity
txn_counts.columns = ['is_loyalty_member', 'transaction_count']

print(txn_counts)

## Question 2

What is the average transaction value for loyalty program members and non-members during July 2024? Use this to identify which group has a higher average transaction value.

In [ ]:
# If columns might contain spaces or different casing, normalize them first
fct_transactions = fct_transactions.rename(columns=lambda c: c.strip().lower().replace(' ', '_'))
dim_customers   = dim_customers.rename(columns=lambda c: c.strip().lower().replace(' ', '_'))

# Ensure required columns exist after normalization
required_txn_cols = {'transaction_id', 'customer_id', 'transaction_date', 'transaction_value'}
required_cust_cols = {'customer_id', 'is_loyalty_member'}
missing_txn = required_txn_cols - set(fct_transactions.columns)
missing_cust = required_cust_cols - set(dim_customers.columns)
if missing_txn or missing_cust:
    raise KeyError(f"Missing columns — transactions: {missing_txn}, customers: {missing_cust}")

# Parse dates
fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'], errors='coerce')

# Filter for July 2024
july_txn = fct_transactions[
    (fct_transactions['transaction_date'].dt.year == 2024) &
    (fct_transactions['transaction_date'].dt.month == 7)
].copy()

# Merge with customer loyalty info
merged_df = july_txn.merge(dim_customers[['customer_id', 'is_loyalty_member']], on='customer_id', how='left')

# Compute average transaction value by loyalty membership
avg_txn_value = (
    merged_df.groupby('is_loyalty_member', dropna=False)['transaction_value']
    .mean()
    .reset_index(name='avg_transaction_value')
)

print(avg_txn_value)

## Question 3

Determine the percentage difference in average transaction value between loyalty program members and non-members for July 2024.

In [ ]:
# Convert transaction_date to datetime
fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'])

# Filter for July 2024 transactions
july_transactions = fct_transactions[
    (fct_transactions['transaction_date'].dt.year == 2024) &
    (fct_transactions['transaction_date'].dt.month == 7)
]

# Merge transactions with customer loyalty info
merged_df = july_transactions.merge(dim_customers, on='customer_id', how='left')

# Calculate average transaction value for each group
avg_values = merged_df.groupby('is_loyalty_member')['transaction_value'].mean()

# Calculate percentage difference
percentage_diff = ((avg_values[True] - avg_values[False]) / avg_values[False]) * 100

print("Average Transaction Values (July 2024):")
print(avg_values)
print(f"\nPercentage Difference: {percentage_diff:.2f}%")

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)